In [1]:
def collapse(df):
    time_df=compute_time_features(df)
    freq_df=compute_freq_features(df)
    return pd.concat([time_df, freq_df], axis=1)

In [2]:
def compute_freq_features(df):
    data_total = pd.DataFrame()
    for column in df.columns:
        temp = find_fft_points(df[column], column)
        data_total = pd.concat([data_total, temp], axis=1)
    return data_total

In [3]:
def find_fft_points(data, name):
    (indices_peaks, peaks) = compute_peaks(data)
    columns_x=[name + "X#1", name + "X#2", name + "X#3", name + "X#4", name + "X#5"]
    columns_y=[name + "P#1", name + "P#2", name + "P#3", name + "P#4", name + "P#5"]
    x_p = pd.DataFrame(data=indices_peaks).T
    x_p.columns = columns_x
    y_p = pd.DataFrame(data=peaks).T
    y_p.columns = columns_y
    tot_p = pd.concat([x_p, y_p], axis=1)
    return tot_p

In [4]:
def compute_peaks(data):
    N = len(data)
    T= 1/50
    yf=fft(np.asarray(data))
    xf=fftfreq(N,T)[:N//2]
    yf=2.0/N*np.abs(yf[0:N//2])

    max_peak_height=0.001*np.nanmax(yf)
    indices_peaks=detecta.detect_peaks(yf, mph=max_peak_height)
    peaks = yf[indices_peaks]
    return (indices_peaks[0:5], peaks[0:5])

In [5]:
def find_time_features(data, name):
    columns = [name + "_mean", name + "_std", name + "_range", name + "_IRQ"]
    properties = [np.mean(data), np.std(data), np.max(data) - np.min(data),
                  np.quantile(data, 0.75) - np.quantile(data, 0.25)]
    d = pd.DataFrame(data=properties).T
    d.columns = columns
    return d

In [6]:
from matplotlib import pyplot as plt
import scipy as sc
import numpy as np
from numpy.fft import fft, ifft, fftfreq
import detecta

In [7]:
def compute_time_features(df):
    data_total = pd.DataFrame()
    for column in df.columns:
        temp = find_time_features(df[column], column)
        data_total = pd.concat([data_total, temp], axis=1)
    return data_total

In [8]:
import struct

def recv_msg(sock):
    # Read message length and unpack it into an integer
    msglen = int(sock.recv(8))
    if not msglen:
        return None
    # Read the message data
    return recvall(sock, msglen)

def recvall(sock, n):
    # Helper function to recv n bytes or return None if EOF is hit
    data = bytearray()
    while len(data) < n:
        packet = sock.recv(n - len(data))
        if not packet:
            return None
        data.extend(packet)
    return data

In [ ]:
import socket, sys, pandas as pd
HOST = ''
PORT = 1060
ADDR =(HOST,PORT)
def server(p):
    s = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
    s.setsockopt(socket.SOL_SOCKET,socket.SO_REUSEADDR,1)
    s.bind(ADDR)
    s.listen(1)
    print ('the server is listening at',s.getsockname())
    ss,sockname = s.accept()
    i=0
    while True:
        #accept the complete message
        msg=recv_msg(ss).decode()
        samples=msg.split("|")
        if len(samples) == 6:
            lin_acc_x=list(map(float, samples[0][1:len(samples[0])-1].split(", ")))
            lin_acc_y=list(map(float, samples[1][1:len(samples[1])-1].split(", ")))
            lin_acc_z=list(map(float, samples[2][1:len(samples[2])-1].split(", ")))
            rot_x=list(map(float, samples[3][1:len(samples[3])-1].split(", ")))
            rot_y=list(map(float, samples[4][1:len(samples[4])-1].split(", ")))
            rot_z=list(map(float, samples[5][1:len(samples[5])-1].split(", ")))
            df_len=min([len(lin_acc_x), len(lin_acc_y), len(lin_acc_z), len(rot_x), len(rot_y), len(rot_z)])

            df=pd.DataFrame(columns=["rotationRate.x", "rotationRate.y", "rotationRate.z", "userAcceleration.x", "userAcceleration.y", "userAcceleration.z"])
            df["rotationRate.x"]=rot_x[0:df_len]
            df["rotationRate.x"]=df["rotationRate.x"]*2
            df["rotationRate.y"]=rot_y[0:df_len]
            df["rotationRate.y"]=df["rotationRate.y"]*2
            df["rotationRate.z"]=rot_z[0:df_len]
            df["rotationRate.z"]=df["rotationRate.z"]*2
            df["userAcceleration.x"]=lin_acc_x[0:df_len]
            df["userAcceleration.x"]=df["userAcceleration.x"]/5
            df["userAcceleration.y"]=lin_acc_y[0:df_len]
            df["userAcceleration.y"]=df["userAcceleration.y"]/5
            df["userAcceleration.z"]=lin_acc_z[0:df_len]
            df["userAcceleration.z"]=df["userAcceleration.z"]/5

            collapsed_df=collapse(df)
            prediction=p.predict(collapsed_df)
            print(prediction)

    ss.close()

dataframe_file=pd.read_csv("collapsed_datasets/dataset_peaks_honor.csv")
dataframe_file=dataframe_file.drop("Unnamed: 0", axis=1)
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest
estimators = [('scaling', StandardScaler()),
              ('sampling', SMOTE()),
              ('feature-selection',SelectKBest(k=20)),
              ('clf', KNeighborsClassifier(n_neighbors=5))]

pipe=Pipeline(estimators)
pipe.fit(dataframe_file.drop('class', axis=1), dataframe_file['class'])

server(pipe)


the server is listening at ('0.0.0.0', 1060)
['standing']
['sitting']
['sitting']
['sitting']
['sitting']
